<a href="https://colab.research.google.com/github/dlagur/ProjectHanium/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%B6%94%EC%B6%9C_%EC%88%98%EC%A0%95_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.2 MB/s 
     |████████████████████████████████| 358 kB 48.9 MB/s 
     |████████████████████████████████| 139 kB 55.7 MB/s 
     |████████████████████████████████| 55 kB 850 kB/s 
     |████████████████████████████████| 4.1 MB 37.6 MB/s 
     |████████████████████████████████| 58 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://s

In [2]:
#크롤링시 필요한 라이브러리 불러오기
from selenium import webdriver
from urllib.request import urlopen
import requests
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import re

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('incognito')
driver = webdriver.Chrome('chromedriver', options=options)

# url 바꾸기 ( page 숫자 제외 )

In [25]:
def get_page_urls(end_page):
  urls= []
  for i in range(1, end_page+1): #page 번호를 제외한 url은 바꿔줘야함!!!
      url = ''+str(i)
      urls.append(url)
  return urls  

In [26]:
content_urls = []
for k in get_page_urls(10):

    response = requests.get(k) # 응답
    soup = BeautifulSoup(response.text, 'html.parser') # 응답을 html 문서를 파싱

    content = soup.select('p.tit > a')

    content_urls.append(content)

In [27]:
content_urls = sum(content_urls, []) #2차원 리스트를 1차원으로 변환

In [28]:
jobkorea_urls = []
for i in content_urls:

  link = 'https://www.jobkorea.co.kr'+ i.attrs['href']
  jobkorea_urls.append(link)
    
print(jobkorea_urls)     

['https://www.jobkorea.co.kr/starter/PassAssay/View/239003?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/239004?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238598?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238607?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238520?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238071?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238083?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/starter/PassAssay/View/238084?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10021&Pass_An_Stat=0', 'https://www.jobkorea.co.kr/sta

In [29]:
len(jobkorea_urls)

200

In [ ]:
###다음 기사 본문 및 제목 가져오기###


questions = []
contents = []
companys = []

for i in jobkorea_urls:
    
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html5lib') # 응답을 html 문서를 파싱하고 직렬화
    
    # 문항 가져오기
    question = soup.select('dt.on')
    # list합치기
    question = ''.join(str(question))
    # html태그제거
    question = re.sub('(<([^>]+)>)', '', question)
    question = re.sub(' ', '', question)
    question = re.sub('\n', '', question)
    questions.append(question)



    # 답변 가져오기
    content = soup.select('dl.qnaLists > dd.show > div.tx')
    # list합치기
    content = ''.join(str(content))
    #html태그제거 및 텍스트 다듬기
    content = re.sub('(<([^>]+)>)', '', content)
    content = re.sub('\n', '', content)
    contents.append(content)

    #회사명 가져오기
    company = soup.select('div.selfTopBx > div.viewTitWrap > h2.hd > strong > a')
    # list합치기
    company = ''.join(str(company))
    #html태그제거 및 텍스트 다듬기
    company = re.sub('(<([^>]+)>)', '', company)
    companys.append(company)

  
print(questions)
print(contents)
print(companys)

# text(직무명)

In [31]:
import pandas as pd

text = '건설' #파일 이름 바꾸기

resume_df = pd.DataFrame({'question': questions,'answer': contents, 'company':companys})

resume_df.to_csv('%s.csv'%text, index=False,encoding='utf-8-sig') 

In [ ]:
resume_df

파일 합칠때 코드